# 소비 패턴 기반 패션 추천 LLM 서비스



1.   기술 스텍 및 모델 선정


*   입력 처리 모듈
*   모델 처리 모듈(MLLM)
*   추천 생성 모듈

> 이미지 및 언어 모델 : CLIP / GPT

> 추천 알고리 : Cosine Similarity, k-Nearest Neighbor (k-NN), Ranking Algorithms


2.   아키텍처 설계
*    입력단계 : 이미지 업로드 / 텍스트 입력 (구매한 물품들의 특징에 대한 설명(브랜드,스타일,색상)
*    추천 prompt 생성 : 이미지임베딩과 텍스트 임베딩을 결합하여 추천 prompt 생성


> ex) 구매한 물품 이미지들 (예: "청바지, 블라우스, 운동화")
텍스트 프롬프트 (예: "운동복을 좋아하는 30대 남성 고객에게 어울리는 패션 아이템을 추천해 주세요")

*    출력 단계 : 추천된 패션 아이템 목록을 이미지 및 텍스트 형태 출력

3. 서비스 개발 흐름


>1. 사용자가 전신 이미지와 구매한 아이템 이미지를 업로드.
2. CLIP 모델로 각 이미지를 임베딩 벡터로 변환.
3. GPT 모델로 패션 아이템 추천 텍스트 생성.
4. 추천된 패션 아이템 이미지 임베딩을 CLIP 모델로 추출.
5. 사용자 이미지와 추천된 아이템 이미지 간의 유사도 계산.
6. 가장 유사한 아이템을 추천하고, 텍스트 설명을 함께 제공.


4. 생각해보기


> 나중에 라벨링을 어떻게 할지 고민해봐야.?

> 화장품 추천해주는 것도 좋다고 멘토링시간에 얘기해주심
  * 얼굴 관련 기존 데이터셋이 많음

> 외국어 학습 도우미, 맞춤형 여행지 및 맛집 추천, 역사적 사건 시뮬레이터?

## [MYCODE] OpenAI 사전준비

사용 라이브러리

In [1]:
pip install transformers torch torchvision openai pillow numpy scikit-learn fastapi uvicorn matplotlib boto3 sentence-transformers faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.7 MB/s eta 0:00:00


In [2]:
import base64
from openai import OpenAI
from google.colab import userdata

api_key = userdata.get('api_key')

client = OpenAI(api_key=api_key)

## [MYCODE] Model load

In [29]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import openai

# CLIP 로드
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32") #이미지와 텍스트 데이터 전처리

# GPT-4 API 호출
def generate_recommendation(prompt):
  messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "system", "content": "You are an excellent stylist."},
    {"role": "user", "content": prompt+"\n## 한글로 답변해주세요"}
]
  response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    ) #response['choices'][0]['message']['content']
  return response.choices[0].message.content


clip_model 주요 반환 값

* image_embeds : 이미지가 고유하게 나타내는 의미를 벡터로 표현한 값

* text_embeds : 텍스트의 의미를 고유한 벡터 표현으로 변환한 값

* logits_per_image : 입력 이미지와 텍스트 간 유사성 점수

* logits_per_text : 입력 텍스트와 이미지 간 유사성 점수

In [ ]:
clip_model

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

clip_processor 주요 반환 값

*   input_ids: 텍스트의 토큰 ID.
*   attention_mask: 텍스트의 유효 토큰 위치를 나타내는 마스크.
*   pixel_values: 전처리된 이미지 텐서.

In [ ]:
clip_processor

CLIPProcessor:
- image_processor: CLIPImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "CLIPImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

- tokenizer: CLIPTokenizerFast(name_or_path='openai/clip-vit-base-patch32', vocab_size=49408, model_max_length=77, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	49406: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, spec

## [MYCODE] 사진 데이터 준비

In [4]:
#구글 드라이브 마운트
from google.colab import drive
drive.flush_and_unmount()  # 이전 인증 삭제
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [5]:
#구매한 아이템 이미지 파일 경로 리스트
image_path = ['/content/drive/MyDrive/photo/jacket.webp','/content/drive/MyDrive/photo/trucker.webp']
# 텍스트를 결합한 프롬프트
text_prompts = [
    "Suggest pants that go well with these jacket."
]

## [MYCODE] 이미지,텍스트 임베딩 처리

In [6]:
def process_image(image_paths,text_prompt):
    """4
    여러 이미지 경로를 입력으로 받아 처리. 단일 이미지도 리스트로 처리됨.

    Args:
        image_paths (list): 이미지 경로 리스트.

    Returns:
        torch.Tensor: 이미지 임베딩 텐서. 배치 형태로 반환.
    """
    # 이미지 로드
    images = [Image.open(path) for path in image_paths]

    # 이미지 텍스트 전처리
    inputs = clip_processor(text=text_prompts, images=images, return_tensors="pt", padding=True)

    image_tensor = inputs["pixel_values"]
    #print(image_tensor.shape) # torch.Size([batch_size, num_channels, height, width])

    outputs = clip_model(**inputs)

    return outputs

In [7]:
import torch
from torch.nn.functional import cosine_similarity

outputs = process_image(image_path,text_prompts)
print(outputs.keys())

 # 이미지와 텍스트의 임베딩 추출
image_embeds = outputs.image_embeds  # 이미지 임베딩
text_embeds = outputs.text_embeds    # 텍스트 임베딩

print("Image embedding shape:", image_embeds.shape , image_embeds)
print("Text embedding shape:", text_embeds.shape, text_embeds)

# 이미지들의 임베딩의 평균 계산
combined_embedding = image_embeds.mean(dim=0)  # (3,)

# 텍스트와의 유사도 계산
similarity = cosine_similarity(combined_embedding.unsqueeze(0), text_embeds.unsqueeze(0), dim=1)
print("결합된 임베딩 유사도:", similarity)

top_matches = torch.argsort(similarity, descending=True)

# print("top_matches:", top_matches)

odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])
Image embedding shape: torch.Size([2, 512]) tensor([[-0.0296,  0.0002,  0.0146,  ...,  0.0374,  0.0053,  0.0307],
        [-0.0024,  0.0255,  0.0357,  ...,  0.0951,  0.0133,  0.0527]],
       grad_fn=<DivBackward0>)
Text embedding shape: torch.Size([1, 512]) tensor([[-2.8240e-03,  5.2022e-02, -6.9584e-03, -9.1824e-03, -1.2933e-02,
         -4.4345e-03, -6.2870e-02, -2.2477e-02, -1.3359e-03, -2.3438e-02,
         -3.5738e-03,  1.4213e-02, -1.3139e-02, -2.2988e-02,  2.2256e-02,
          6.1014e-03,  2.8493e-02, -8.9735e-03,  1.5654e-02,  2.0316e-02,
          9.3036e-03,  3.6925e-02,  4.7369e-03, -1.9623e-02,  4.2255e-02,
          6.8918e-03, -2.4085e-02,  3.2735e-02, -1.1243e-02,  2.1658e-02,
         -2.6321e-02, -6.5498e-03,  2.4495e-02,  1.0221e-02,  9.5645e-03,
         -1.5969e-02,  9.3841e-03, -3.5463e-02, -1.1167e-02,  4.4552e-02,
         -2.4406e-02, 

## [MYCODE] GPT 모델에 입력할 prompt 생성
> 유사도 계산하여 점수가 제일 높은 이미지를 선택하게 되있으나 input 이미지가 2개뿐으로 전체 를 넣어주는걸로 진행

In [33]:
# 텍스트 프롬프트에 이미지 임베딩을 넣어준다
embedding_text = ', '.join([str(x) for x in image_embeds])
#print("image_embedding_text : ", embedding_text)
# 텍스트 임베딩을 문자열로 변환 (이 방법은 실제로 매우 단순화된 형태입니다)
text_prompt = f"Given the image embedding: {embedding_text},\nPlease recommend some pants that would match the jacket photo."
print(text_prompt)

Given the image embedding: tensor([-2.9559e-02,  2.3442e-04,  1.4630e-02,  1.8316e-03, -5.0913e-03,
         6.5859e-03, -5.8771e-02,  1.5978e-02,  2.9565e-02,  2.2867e-02,
        -5.2097e-03,  2.0580e-03, -1.8769e-02,  3.3024e-03,  2.7587e-02,
        -2.2074e-02, -3.1574e-02,  6.0606e-03, -1.1207e-02, -2.2085e-03,
        -1.5251e-02,  1.1632e-02,  3.6212e-02, -4.5900e-02,  3.3719e-02,
         1.5531e-02,  8.9285e-03,  1.0745e-02,  3.1603e-03, -1.3570e-02,
         3.7793e-02, -1.6377e-02, -2.7728e-02,  9.1010e-04,  2.5790e-02,
         3.9760e-02,  3.6465e-03,  5.3960e-02,  8.1518e-03,  1.7742e-01,
        -2.5719e-02,  1.0741e-02, -4.1633e-04,  2.5294e-02,  1.7934e-03,
        -7.8408e-02,  2.6420e-02,  2.5408e-02,  1.6493e-02, -2.2836e-03,
         9.2802e-02, -2.8382e-02,  1.5447e-02,  4.6112e-03, -4.5208e-02,
         1.9615e-02,  3.1397e-02,  1.4522e-02, -1.6510e-02,  1.6095e-02,
         1.2772e-02,  1.5448e-02,  4.8830e-02, -7.5552e-03, -5.5758e-02,
        -2.8686e-03, -5.

## [MYCODE] 최종 결과

In [34]:
recommendations = generate_recommendation(text_prompt)
print(recommendations)

이 전자 사진의 이미지 임베딩을 고려했을 때, 다음과 같은 스타일의 바지를 추천합니다:

1. **슬림 핏 팬츠**: 이미지 임베딩에서 추출된 정보가 모던하고 세련된 느낌을 준다면, 슬림 핏 팬츠가 잘 어울릴 것입니다. 간결하고 깔끔한 라인이 전체적인 룩의 일관성을 유지해 줄 것입니다.

2. **네이비 또는 다크 컬러 팬츠**: 네이비 또는 짙은 색상의 바지는 다채로운 색상의 자켓과도 잘 어울리며, 고급스러운 느낌을 더해 줄 수 있습니다.

3. **카키 또는 올리브 그린 팬츠**: 좀 더 캐주얼하면서도 트렌디한 느낌을 원한다면, 카키 또는 올리브 그린 색상의 팬츠도 고려해볼 만합니다.

4. **청바지**: 자켓의 스타일에 따라 다르겠지만, 약간 캐주얼하면서도 멋스럽게 연출하고 싶다면 진한 색상의 슬림 핏 청바지도 좋은 선택이 될 수 있습니다.

이 추천들은 제시된 이미지 임베딩을 기반으로 한 것이며, 최종적인 선택은 자켓의 구체적인 디자인과 사용자의 개인 스타일에 따라 달라질 수 있습니다.
